In [1]:
import torch as T
import albumentations as A
import cv2
import numpy as np
import os
import random
from tqdm import tqdm


In [2]:
if T.cuda.is_available():
    device=T.device("cuda")
else:
    device=T.device("cpu")

print(device)

cuda


In [3]:
# ---------------------- Augmentation Pipelines -----------------------
env_aug = A.Compose([
    A.OneOf([
        A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), src_radius=200, p=0.5),
        A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=3, p=0.5),
        A.RandomFog(fog_coef_range=(0.2, 0.4), p=0.5)
    ], p=1),
    A.Resize(384, 512)
], additional_targets={'crop_mask': 'mask', 'vegetation_mask': 'mask'})

geo_aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Affine(translate_percent=0.05, scale=(0.9, 1.1), p=0.5),
    A.ElasticTransform(alpha=300, sigma=10, interpolation=cv2.INTER_NEAREST,
                       mask_interpolation=cv2.INTER_NEAREST, same_dxdy=True,
                       border_mode=cv2.BORDER_CONSTANT, p=0.5),
    A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=0.5),
    A.Resize(384, 512)
], additional_targets={'crop_mask': 'mask', 'vegetation_mask': 'mask'})

# ---------------------- Folders -----------------------
# Hardcoded full path to IMAGE512x384 (update if needed)
folder_prefix = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384"

folders = ["train", "test", "validation"]
new_folders = ["train_new", "test_new", "validation_new"]
mask_folders = ["trainlabel", "testlabel", "validationlabel"]
new_mask_folders = ["trainlabel_new", "testlabel_new", "validationlabel_new"]
vegetation_mask_folders = ["train", "test", "val"]

img_ext = ".jpg"
mask_ext = ".png"

# ---------------------- Helper Functions -----------------------
def get_image_files(folder, extension):
    return [os.path.join(folder, f) for f in os.listdir(folder)
            if os.path.splitext(f)[1].lower() == extension]

def load_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Failed to load {image_path}")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def load_mask(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise ValueError(f"Failed to load {mask_path}")
    return mask

def save_image(path, image_rgb):
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite(path, image_bgr)

def save_mask(path, mask):
    cv2.imwrite(path, mask)

# ---------------------- Main Loop -----------------------
for folder_div, new_folder_div, mask_folder_div, veg_mask_folder_div, new_mask_folder_div, new_veg_mask_folder_div in zip(
    folders, new_folders, mask_folders, vegetation_mask_folders, new_mask_folders, vegetation_mask_folders):

    folder = os.path.join(folder_prefix, folder_div)
    new_folder = os.path.join(folder_prefix, new_folder_div)
    mask_folder = os.path.join(folder_prefix, mask_folder_div)
    veg_mask_folder = os.path.join("Vegetation_Masks", veg_mask_folder_div)
    new_mask_folder = os.path.join(folder_prefix, new_mask_folder_div)
    new_veg_mask_folder = os.path.join("Vegetation_Masks", new_veg_mask_folder_div)

    # Check if required folders exist
    required_folders = [
        (folder, "Image folder"),
        (mask_folder, "Crop mask folder"),
        (veg_mask_folder, "Vegetation mask folder")
    ]

    missing_folders = [desc for path, desc in required_folders if not os.path.isdir(path)]

    if missing_folders:
        print(f"Skipping {folder_div} - Missing: {', '.join(missing_folders)}")
        continue

    os.makedirs(new_folder, exist_ok=True)
    os.makedirs(new_mask_folder, exist_ok=True)
    os.makedirs(new_veg_mask_folder, exist_ok=True)

    print(f"\n📁 Processing folder: {folder_div}")
    image_paths = get_image_files(folder, img_ext)
    image_paths.sort()

    images, crop_masks, veg_masks, names = [], [], [], []

    # Load original data
    for path in image_paths:
        image = load_image(path)
        basename = os.path.splitext(os.path.basename(path))[0]

        crop_mask_path = os.path.join(mask_folder, basename + mask_ext)
        crop_mask = load_mask(crop_mask_path)

        veg_mask_path = os.path.join(veg_mask_folder, basename + mask_ext)
        veg_mask = load_mask(veg_mask_path)

        images.append(image)
        crop_masks.append(crop_mask)
        veg_masks.append(veg_mask)
        names.append(basename)

    orig_count = len(images)
    all_aug_images, all_aug_crop_masks, all_aug_veg_masks, all_aug_names = [], [], [], []

    # Environmental Augmentations
    for i, (image, crop_mask, veg_mask, basename) in enumerate(tqdm(
        list(zip(images, crop_masks, veg_masks, names)), desc="🌦️ Environmental Aug")):

        augmented = env_aug(image=image, crop_mask=crop_mask, vegetation_mask=veg_mask)
        all_aug_images.append(augmented['image'])
        all_aug_crop_masks.append(augmented['crop_mask'])
        all_aug_veg_masks.append(augmented['vegetation_mask'])
        all_aug_names.append(f"{orig_count + i + 1}_env")

    # Geometric Augmentations
    current_max_id = orig_count * 2
    for i, (img, crop_msk, veg_msk, _) in enumerate(tqdm(
        list(zip(images + all_aug_images,
                 crop_masks + all_aug_crop_masks,
                 veg_masks + all_aug_veg_masks,
                 names + all_aug_names)), desc="🌀 Geometric Aug")):

        augmented = geo_aug(image=img, crop_mask=crop_msk, vegetation_mask=veg_msk)
        all_aug_images.append(augmented['image'])
        all_aug_crop_masks.append(augmented['crop_mask'])
        all_aug_veg_masks.append(augmented['vegetation_mask'])
        all_aug_names.append(f"{current_max_id + i + 1}_geo")

    combined = list(zip(images + all_aug_images,
                        crop_masks + all_aug_crop_masks,
                        veg_masks + all_aug_veg_masks,
                        names + all_aug_names))
    random.shuffle(combined)

    
    # Save all data
    for i, (img, crop_msk, veg_msk, _) in enumerate(tqdm(combined, desc="💾 Saving")):
        new_name = f"{i + 1}_image"

        save_image(os.path.join(new_folder, new_name + img_ext), img)
        save_mask(os.path.join(new_mask_folder, new_name + mask_ext), crop_msk)
        save_mask(os.path.join(new_veg_mask_folder, new_name + mask_ext), veg_msk)

    print(f"✅ {folder_div}: Original = {orig_count} | Final Augmented = {len(combined)} (4x)")



📁 Processing folder: train


💾 Saving: 100%|███████████████████████████████████████████████████████████████████| 1600/1600 [00:18<00:00, 86.58it/s]


✅ train: Original = 400 | Final Augmented = 1600 (4x)

📁 Processing folder: test


💾 Saving: 100%|███████████████████████████████████████████████████████████████████| 1200/1200 [00:12<00:00, 93.76it/s]


✅ test: Original = 300 | Final Augmented = 1200 (4x)

📁 Processing folder: validation


💾 Saving: 100%|█████████████████████████████████████████████████████████████████████| 352/352 [00:03<00:00, 95.47it/s]

✅ validation: Original = 88 | Final Augmented = 352 (4x)
